In [1]:
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib qt5
from scipy.io import loadmat, savemat

import random


In [2]:
epochs = loadmat('C:/Users/annma/Documents/IAI MSU/BCI-LLM/open-datasets/archive_kaggle/epochs/epochs_S01.mat')
fs = 250

In [3]:
tar_eeg = epochs['tar_eeg']
print(tar_eeg.shape)

nontar_eeg = epochs['nontar_eeg']
print(nontar_eeg.shape)

tar_letters = epochs['tar_letters']
print(tar_letters.shape)

(700, 250, 8)
(3500, 250, 8)
(700,)


In [4]:
unique_targets = np.unique(epochs['tar_letters'])
print(unique_targets)

['A' 'C' 'E' 'G' 'I' 'J' 'K' 'M' 'N' 'O' 'Q' 'R' 'S' 'T' 'U' 'Y']


In [5]:
n_ch = tar_eeg.shape[-1]

# generate sequence of letters 
# take original targets
targets2type = list(tar_letters)
# shuffle original targets
random.shuffle(targets2type)
# set number of targets we want to spell
n_letters = 30
# get slice of shuffled targets 
targets2type = targets2type[:n_letters]
# print(len(targets2type))

# min and max numbers of how many times we want to "whitness" each target letter
n_gazes_minmax = [3, 6]

# min and max number of seconds of a nontarget brain activity
# we want to add in between each target sample
add_timepoints_minmax = [0.2, 1.5]
nontar_eeg_cat = np.concatenate(nontar_eeg, axis=0)
# nontar_eeg_cat.shape # 875000,8
nontar_maxidx = int(add_timepoints_minmax[1] * fs)
# print(nontar_maxidx)


new_dataset = []
new_tars = []

for letter2type in targets2type:
	# print(letter2type)
	
	# choose n gazes as a random number in n_gazes_minmax
	n_gazes = random.randint(n_gazes_minmax[0], n_gazes_minmax[1])
	# get n gazes random indices of eeg to add
	idx_data = list(np.where(tar_letters==letter2type)[0])
	# get seq eeg to add
	add_tar_seq = tar_eeg[random.sample(idx_data, n_gazes),:,:]

	new_data2add = []
	new_tar2add = []
	for i in range(len(add_tar_seq)):
		
		# get nontarget eeg seq to add
		# start idx
		nontar_idx = random.randint(0, len(nontar_eeg_cat)-nontar_maxidx-1)
		# len of nontar seq
		nontar_len = random.randint((add_timepoints_minmax[0]*fs), (add_timepoints_minmax[1]*fs))
		# print(nontar_idx)
		# print(nontar_eeg_cat[nontar_idx:nontar_idx+nontar_len,:].shape)
		
		tar_nontar_seq = np.append( # tar seq +-> nontar seq
				add_tar_seq[i], 
				nontar_eeg_cat[nontar_idx:nontar_idx+nontar_len,:],
				axis=0)
		# print(tar_nontar_seq.shape)
		
		# get list of targets seq
		tars = list(np.repeat(letter2type, add_tar_seq[i].shape[0]))
		# print(tars)
		# get list of nontargets seq
		nontars = list(np.repeat('None', nontar_len))
		
		new_data2add.append(tar_nontar_seq)
		new_tar2add.append(tars+nontars)

	new_dataset.append(np.concatenate(new_data2add, axis=0))
	new_tars.append(np.concatenate(new_tar2add, axis=0))

new_dataset = np.concatenate(new_dataset, axis=0)
new_tars = np.concatenate(new_tars, axis=0)

print(new_dataset.shape)
print(new_tars.shape)

(59702, 8)
(59702,)


In [6]:
tars_dict = dict(zip(np.unique(tar_letters), np.arange(1, len(np.unique(tar_letters))+1)))
tars_dict['None'] = 0
display(tars_dict)

{'A': 1,
 'C': 2,
 'E': 3,
 'G': 4,
 'I': 5,
 'J': 6,
 'K': 7,
 'M': 8,
 'N': 9,
 'O': 10,
 'Q': 11,
 'R': 12,
 'S': 13,
 'T': 14,
 'U': 15,
 'Y': 16,
 'None': 0}

In [7]:
tars_int = [tars_dict[letter] for letter in new_tars]
print(tars_int[:3000:100])

[14, 14, 14, 0, 14, 14, 14, 0, 0, 0, 0, 14, 14, 0, 0, 0, 0, 14, 14, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 0]


In [ ]:
# %matplotlib qt5
plt.figure(figsize=(9,2.5))
plt.plot(tars_int[2000:16000])
plt.legend(['1-16: letters \n 0: None'], loc='upper right')

In [10]:
savemat('C:/Users/annma/Documents/IAI MSU/BCI-LLM/open-datasets/archive_kaggle/radom_samples/sampled.mat',
        {'eeg': new_dataset,
        'targets': new_tars,
        'fs': fs
        }
        )